In [4]:
import math,csv, random, collections
import numpy as np
import pickle
import seaborn as sns
import matplotlib.patches as mpatches
import random, string

import imp, sys
sys.path.insert(0, '../')
import utils, algorithms
import h5py, imp

import matplotlib.pyplot as plt
from IPython.display import display, HTML
from multiprocessing import Pool
from multiprocessing import Process, Queue
imp.reload(utils)
imp.reload(algorithms)
        
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
sns.set_style('dark')
color_p = sns.color_palette("hls", 10*2)

In [5]:
def BP(M, n_steps, filename):
    n = M.shape[0]
    BP_matrix = np.zeros((2, n, n)) #BP_matrix[i] will store the state of BP in step i
    BP_matrix[0] = M.copy() #First state of BP_matrix is the data matrix itself
    accuracy = np.zeros(n_steps) #Stores accuracy after each step
    
    #True answer
    true_answer = np.zeros((n,2))
    true_answer[:,0] = np.roll(np.arange(n),1)
    true_answer[:,1] = np.roll(np.arange(n),-1)
    true_answer = true_answer.astype(int)

    for l in range(1, n_steps):
        for i in range(n):
            for j in range(n):
                row = BP_matrix[(l-1)%2][:,i].copy()
                row[j] = np.inf
                BP_matrix[l%2][i][j] = M[i][j] - np.partition(row, 1)[1]
                
        BP_matrix[l%2][np.diag_indices_from(BP_matrix[l%2])] = np.inf
        
        M_final = BP_matrix[l%2]
        M_final[np.diag_indices_from(M_final)] = np.inf
        answer = M_final.argsort(axis=1)[:,:2]
        answer.sort(axis=1)
        answer[0] = answer[0][::-1]
        answer[-1] = answer[-1][::-1]
        accuracy[l] = np.mean(answer==true_answer)
        if accuracy[l] == 1:
            accuracy[l:] = accuracy[l]
            break
        if l > 75 and abs(np.mean(accuracy[l-25:l]) - np.mean(accuracy[l-50:l-25])) < 0.001: 
            accuracy[l:] = np.mean(accuracy[l-20:l])
            break
    print "BP", l , "Steps"
    output = ','.join(np.array(accuracy[1:]).astype(str))
    output = str(n)+","+str(mu)+","+str(l)+","+output
    fd = open(filename,'a')
    fd.write(output+"\n")
    fd.close()




def LP(data, file):
    n = data.shape[0]
    model , t      = algorithms.Solve_LP(data)
    answer         = np.array(model.X).reshape(n,n)
    correct_answer = algorithms.correct_data_answer(data,k=1)
           
    error = (np.where(correct_answer-answer!=0)[0].shape[0]+0.0)/(2*n)
    fp = (np.where(correct_answer-answer==-1)[0].shape[0]+0.0)/(2*n)
    point5 = (np.where(answer==0.5)[0].shape[0]+0.0)/(2*n)
    if file != None:
        fd = open(file,'a')
        fd.write(', '.join([str(n), str(mu), str(error), str(fp), str(point5),"\n"]))
        fd.close()

In [ ]:
n = 10
it_limit = np.sqrt(4*np.log(n))
mg_limit = np.sqrt(6*np.log(n))
sg_limit = np.sqrt(8*np.log(n))
mu_array1 = np.linspace(it_limit*.8,it_limit*1.2,8)
mu_array2 = np.linspace(it_limit*.95,it_limit*1.05,8)
mu_array = np.concatenate([mu_array1, mu_array2])
mu_array[np.abs(mu_array-it_limit).argmin()] = it_limit
mu_array[np.abs(mu_array-sg_limit).argmin()] = sg_limit

mu_array.sort()
# M = M.astype(int)
print "it = ", it_limit
print "mg = ", mg_limit
print "sg = ", sg_limit
no_exp = 1

filenameBP = 'data/BP_'+str(n)+'.csv'   
filenameLP = 'data/LP_'+str(n)+'.csv'   

for mu in mu_array:
    print "\n", mu,
    no_process = 32
    pool = Pool(processes=no_process)  
    processes = []
    for exp in range(no_exp):
        print " ", exp+1,
        M = algorithms.Gaussian_matrix(mu=mu, n =n)
        
        #BP
        M[np.diag_indices_from(M)]= -np.inf
        args = (-M, n*10, filenameBP)
        p = Process(target = BP, args = args)
        p.start()
        processes.append(p)
        
        #LP
        M[np.diag_indices_from(M)]= 0
        args = (M, filenameLP)
        p = Process(target = LP, args = args)
        p.start()
        processes.append(p)

    print  "\nJoining Process"
    for j, p in enumerate(processes):
        print j, "\t",
        p.join()

#         accuracy_array[i][exp], _, _ = BP(-M,n)
#         break
#         print "\nSG, MG, LP", accuracy_array[i][exp][1], accuracy_array[i][exp][2], accuracy_array[i][exp][-1]

it =  3.03485425877
mg =  3.71692218885
sg =  4.29193205258

2.42788340702   1 BP 2 Steps
 
Joining Process
0 	1 	
2.60130365037   1 BP 76 Steps
 
Joining Process
0 	1 	
2.77472389373   1 BP 2 Steps
 
Joining Process
0 	1 	
2.88311154583   1 BP 2 Steps
 
Joining Process
0 	1 	
2.92646660667   1 BP 1 Steps
 
Joining Process
0 	1 	
2.94814413709   1 
Joining Process
0 	 BP 76 Steps
1 	
2.96982166751   1 BP 3 Steps
 
Joining Process
0 	1 	
3.03485425877   1 BP 1 Steps
 
Joining Process
0 	1 	
3.05653178919   1 BP 3 Steps
 
Joining Process
0 	1 	
3.09988685003   1 BP 2 Steps
 
Joining Process
0 	1 	
3.12156438045   1 BP 1 Steps
 
Joining Process
0 	1 	
3.14324191087   1 BP 1 Steps
 
Joining Process
0 	1 	
3.18659697171   1 BP 2 Steps
 
Joining Process
0 	1 	
3.29498462381